### Imports 

In [1]:
from sagemaker.tensorflow.serving import TensorFlowModel
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role
from datetime import datetime
import tensorflow as tf
import sagemaker
import time
import boto3
import os

In [2]:
print(f'Using TensorFlow version: {tf.__version__}')
print(f'Using SageMaker version: {sagemaker.__version__}')

Using TensorFlow version: 2.3.0
Using SageMaker version: 2.16.1


### Essentials

In [3]:
role = get_execution_role()
session = boto3.Session()
sagemaker_session = sagemaker.Session()

s3 = session.resource('s3')
TF_FRAMEWORK_VERSION = '2.3.0'
BUCKET = 'tf-mme-892313895307' # USE YOUR ACCOUNT ID OR INITIALS AS SUFFIX
PREFIX = 'cv-models'

## Model 1 - CIFAR-10 Image Classification

### Copy Train & Validation Data to S3

In [4]:
!aws s3 cp ./DATA/CIFAR_10/train s3://{BUCKET}/{PREFIX}/cifar/train --recursive

upload: DATA/CIFAR_10/train/y_train.npy to s3://tf-mme-892313895307/cv-models/cifar/train/y_train.npy
upload: DATA/CIFAR_10/train/X_train.npy to s3://tf-mme-892313895307/cv-models/cifar/train/X_train.npy


In [5]:
!aws s3 cp ./DATA/CIFAR_10/validation s3://{BUCKET}/{PREFIX}/cifar/validation --recursive

upload: DATA/CIFAR_10/validation/y_validation.npy to s3://tf-mme-892313895307/cv-models/cifar/validation/y_validation.npy
upload: DATA/CIFAR_10/validation/X_validation.npy to s3://tf-mme-892313895307/cv-models/cifar/validation/X_validation.npy


In [6]:
!aws s3 cp ./DATA/CIFAR_10/test s3://{BUCKET}/{PREFIX}/cifar/test --recursive

upload: DATA/CIFAR_10/test/y_test.npy to s3://tf-mme-892313895307/cv-models/cifar/test/y_test.npy
upload: DATA/CIFAR_10/test/X_test.npy to s3://tf-mme-892313895307/cv-models/cifar/test/X_test.npy


In [7]:
train_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar/train', 
                            distribution='FullyReplicated', 
                            content_type='npy')
validation_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar/validation', 
                                 distribution='FullyReplicated', 
                                 content_type='npy')
test_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar/test', 
                           distribution='FullyReplicated', 
                           content_type='npy')

In [8]:
inputs = {'train': train_input, 'val': validation_input, 'test': test_input}

### Estimator 1

In [9]:
model_name = 'model-1'
hyperparameters = {'epochs': 30}
estimator_parameters = {'entry_point':'cifar_train.py',
                        'instance_type': 'ml.m5.2xlarge',
                        'instance_count': 1,
                        'model_dir': f'/opt/ml/model',
                        'role': role,
                        'hyperparameters': hyperparameters,
                        'output_path': f's3://{BUCKET}/{PREFIX}/cifar/out',
                        'base_job_name': f'cv-{model_name}',
                        'framework_version': TF_FRAMEWORK_VERSION,
                        'py_version': 'py37',
                        'script_mode': True}
estimator_1 = TensorFlow(**estimator_parameters)

In [ ]:
estimator_1.fit(inputs)

## Model 2 - Sign Language Classification 

### Copy Train & Validation Data to S3

In [ ]:
!aws s3 cp ./DATA/SIGN_LANGUAGE/ s3://{BUCKET}/{PREFIX}/sign_lang --recursive

In [15]:
train_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/sign_lang/train', 
                            distribution='ShardedByS3Key')
val_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/sign_lang/valid', 
                          distribution='ShardedByS3Key')

### Estimator 2

In [16]:
model_name = 'model-2'

hyperparameters = {'epochs': 20}
estimator_parameters = {'entry_point':'sign_language_train.py',
                        'instance_type': 'ml.m5.2xlarge',
                        'instance_count': 1,
                        'hyperparameters': hyperparameters,
                        'model_dir': f'/opt/ml/model',
                        'role': role,
                        'output_path': f's3://{BUCKET}/{PREFIX}/sign_lang/out',
                        'base_job_name': f'cv-{model_name}',
                        'framework_version': TF_FRAMEWORK_VERSION,
                        'py_version': 'py37',
                        'script_mode': True}

estimator_2 = TensorFlow(**estimator_parameters)

In [ ]:
estimator_2.fit({'train': train_input, 'val': val_input})

## Multi-Model Endpoint (MME)

In [18]:
tf_model_1 = estimator_1.model_data
output_1 = f's3://{BUCKET}/{PREFIX}/mme/model-1.tar.gz'

In [19]:
tf_model_2 = estimator_2.model_data
output_2 = f's3://{BUCKET}/{PREFIX}/mme/model-2.tar.gz'

In [20]:
!aws s3 cp {tf_model_1} {output_1} 
!aws s3 cp {tf_model_2} {output_2} 

copy: s3://tf-mme-892313895307/cv-models/cifar/out/cv-model-1-2020-11-27-20-14-12-336/output/model.tar.gz to s3://tf-mme-892313895307/cv-models/mme/model-1.tar.gz
copy: s3://tf-mme-892313895307/cv-models/sign_lang/out/cv-model-2-2020-11-27-20-44-45-298/output/model.tar.gz to s3://tf-mme-892313895307/cv-models/mme/model-2.tar.gz


### Deploy TensorFlow Model-1 as a Multi-Model Endpoint 
<p>https://github.com/aws/deep-learning-containers/blob/master/available_images.md</p>

In [21]:
current_time = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d-%H-%M-%S')
current_time

'2020-11-27-22-42-28'

In [22]:
IMAGE_URI = '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.1-cpu-py37-ubuntu18.04'
model_data_prefix = f's3://{BUCKET}/{PREFIX}/mme/'
model_1 = TensorFlowModel(model_data=output_1, 
                          role=role, 
                          image_uri=IMAGE_URI)

In [23]:
mme = MultiDataModel(name=f'mme-tensorflow-{current_time}',
                     model_data_prefix=model_data_prefix,
                     model=model_1,
                     sagemaker_session=sagemaker_session)

In [24]:
predictor = mme.deploy(initial_instance_count=1,
                       instance_type='ml.m5.2xlarge',
                       endpoint_name=f'mme-tensorflow-{current_time}')

-------------!

### Test MME

In [25]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [26]:
list(mme.list_models())

['model-1.tar.gz', 'model-2.tar.gz']

In [27]:
img_path = './DATA/CIFAR_10/raw_images/truck.png'
img = image.load_img(img_path)
X = image.img_to_array(img)
X = X.astype('float32')/255
X = X.reshape(1, 32, 32, 3)

ValueError: cannot reshape array of size 373248 into shape (1,32,32,3)

In [ ]:
payload = {'instances': X}

In [ ]:
predictor.predict(data=payload, initial_args={'TargetModel': 'model1.tar.gz'})

#### Test Model 2

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
img_path = './DATA/SIGN_LANGUAGE/test/0/IMG_1290.JPG'
img = image.load_img(img_path, target_size=(224, 224))
X = image.img_to_array(img)
X = np.expand_dims(X, axis=0)

In [ ]:
X.shape

In [ ]:
payload = {'instances': X}

In [ ]:
predictor.predict(data=payload, initial_args={'TargetModel': 'model2.tar.gz'})